In [159]:
import sys  
sys.path.insert(0, '..')

from config import Config

import pandas as pd
from sqlalchemy import create_engine

db = create_engine(Config.SQLALCHEMY_DATABASE_URI)

In [193]:
mista = pd.read_sql_query(
    """
    select ockovaci_mista.id ockovaci_misto_id, ockovaci_mista.nazev, okres_id, kraj_id 
    from ockovaci_mista join okresy on ockovaci_mista.okres_id=okresy.id;
    """,
    db
)
# mista

In [195]:
df = pd.read_sql_query("select * from ockovani_distribuce limit 10;", db)
# df

In [196]:
prijato = pd.read_sql_query(
    """
    select ockovaci_misto_id, vyrobce, sum(pocet_davek) prijato
    from ockovani_distribuce 
    where akce = 'Příjem' 
    group by (ockovaci_misto_id, vyrobce);
    """,
    db
)
# prijato

In [197]:
prijato_odjinud = pd.read_sql_query(
    """
    select cilove_ockovaci_misto_id ockovaci_misto_id, vyrobce, sum(pocet_davek) prijato_odjinud
    from ockovani_distribuce 
    where akce = 'Výdej' 
    group by (cilove_ockovaci_misto_id, vyrobce);
    """,
    db
)
# prijato_odjinud

In [198]:
vydano = pd.read_sql_query(
    """
    select ockovaci_misto_id, vyrobce, sum(pocet_davek) vydano
    from ockovani_distribuce 
    where akce = 'Výdej' 
    group by (ockovaci_misto_id, vyrobce);
    """,
    db
)
# vydano

In [199]:
df = pd.read_sql_query("select * from ockovani_spotreba limit 10;", db)
# df

In [200]:
spotreba = pd.read_sql_query(
    """
    select ockovaci_misto_id, vyrobce, sum(pouzite_davky) pouzito, sum(znehodnocene_davky) znehodnoceno
    from ockovani_spotreba 
    group by (ockovaci_misto_id, vyrobce);
    """,
    db
)
# spotreba

In [222]:
mista_key = mista
mista_key['join'] = 0

vyrobci_key = spotreba[['vyrobce']].drop_duplicates()
vyrobci_key['join'] = 0

merged = mista_key.merge(vyrobci_key).drop('join', axis=1)

merged = pd.merge(merged, prijato, how="left")
merged = pd.merge(merged, prijato_odjinud, how="left")
merged = pd.merge(merged, vydano, how="left")
merged = pd.merge(merged, spotreba, how="left")

merged['prijato'] = merged['prijato'].fillna(0).astype('int')
merged['prijato_odjinud'] = merged['prijato_odjinud'].fillna(0).astype('int')
merged['vydano'] = merged['vydano'].fillna(0).astype('int')
merged['pouzito'] = merged['pouzito'].fillna(0).astype('int')
merged['znehodnoceno'] = merged['znehodnoceno'].fillna(0).astype('int')
merged['skladem'] = merged['prijato'] + merged['prijato_odjinud'] - merged['vydano'] - merged['pouzito'] - merged['znehodnoceno']

merged

,ockovaci_misto_id,nazev,okres_id,kraj_id,vyrobce,prijato,prijato_odjinud,vydano,pouzito,znehodnoceno,skladem
0,63c9078b-0701-4da0-821e-4eaa0d9c9968,"Krajská zdravotní a.s., - MNUL pracoviště výst...",CZ0423,CZ042,Moderna,0,0,0,0,0,0
1,63c9078b-0701-4da0-821e-4eaa0d9c9968,"Krajská zdravotní a.s., - MNUL pracoviště výst...",CZ0423,CZ042,Pfizer,0,0,0,0,0,0
2,63c9078b-0701-4da0-821e-4eaa0d9c9968,"Krajská zdravotní a.s., - MNUL pracoviště výst...",CZ0423,CZ042,AstraZeneca,0,0,0,0,0,0
3,989b3700-401f-4700-a684-ae09d348a3c0,Nemocnice Hořovice,CZ0202,CZ020,Moderna,900,0,70,530,0,300
4,989b3700-401f-4700-a684-ae09d348a3c0,Nemocnice Hořovice,CZ0202,CZ020,Pfizer,0,4068,150,3066,30,822
...,...,...,...,...,...,...,...,...,...,...,...
829,98c76ec8-f7d1-474a-927c-d4a4fb2dcd0a,ÚVN Praha - sklad,CZ0100,CZ010,Pfizer,32760,2340,22518,8694,18,3870
830,98c76ec8-f7d1-474a-927c-d4a4fb2dcd0a,ÚVN Praha - sklad,CZ0100,CZ010,AstraZeneca,0,0,0,0,0,0
831,cb0f8a24-3c54-467e-9e67-de01f424ea85,Česká Lípa očkovací místo Crystal,CZ0511,CZ051,Moderna,0,0,0,0,0,0
832,cb0f8a24-3c54-467e-9e67-de01f424ea85,Česká Lípa očkovací místo Crystal,CZ0511,CZ051,Pfizer,0,2874,0,3114,0,-240


In [237]:
prijato = pd.read_sql_query(
    """
    select ockovaci_misto_id, vyrobce, sum(pocet_davek) prijato
    from ockovani_distribuce 
    where akce = 'Příjem' and ockovaci_misto_id in ({}) 
    group by (ockovaci_misto_id, vyrobce);
    """.format(','.join("'" + misto + "'" for misto in mista['ockovaci_misto_id'].tolist())),
    db
)
prijato

,ockovaci_misto_id,vyrobce,prijato
0,6ce2ad8f-b1f6-4252-b6f0-e8029219c17b,Moderna,1100
1,7fd8c9d3-96af-4379-be5c-63849cf5e013,Moderna,400
2,31de7f09-960c-4287-80b4-012568de1703,Moderna,1000
3,44133735-28b0-4c07-a675-f6613a761f8c,Moderna,3000
4,96d6fa7c-79b5-4f4e-9192-9ad7d1a332c7,Pfizer,22230
...,...,...,...
149,4dd9014a-436c-4e8a-aa45-010ad32272f7,Pfizer,24570
150,262721a0-5551-41d6-bc46-06d5aa9d18a8,AstraZeneca,16000
151,96d6fa7c-79b5-4f4e-9192-9ad7d1a332c7,AstraZeneca,21100
152,fe82349e-63db-4623-b6c1-cabd2214c1ee,Moderna,200


In [235]:
','.join("'" + misto + "'" for misto in mista['ockovaci_misto_id'].tolist())

"'63c9078b-0701-4da0-821e-4eaa0d9c9968','989b3700-401f-4700-a684-ae09d348a3c0','586ab361-ba26-4429-b0f3-8ff9831f07b2','df835442-d55c-4653-ad02-23443d6b1700','bbbcf265-9f34-4056-a7e8-f9067fd339a4','c254b811-40e4-4604-8066-feac3dc58c41','20fc59c1-89c7-4221-90bf-4b3fc94027cc','3d5594e1-0aa6-409a-9ced-12eefcf3d981','eb73960f-9644-41ef-8dcc-66d60133d2c6','1ddc573d-b6c1-4585-9729-999f36a0a6f1','be9e7655-6160-4531-816d-62afdf91806b','5c30086c-42d4-4e27-b940-2bf80e739739','07270775-d371-4118-b680-91322bb26b17','cae71614-b96b-4eab-9b3b-41390ff0ae45','19a6168f-de5b-41d1-80f8-1ca39e836f72','c2ec6521-6a37-44c7-a661-1370adbac9c5','95d1157e-5f77-49fe-ae2d-24e5eef3606c','f1b3f2dc-c4d1-4f50-b79a-a48ed0c393d6','da5422b1-8002-456c-aa95-9a26a5e0e622','fafb9109-12b6-4278-87cd-310bd66a356a','a91d8128-c379-456c-b04b-1f1d6aa2da76','ccdcc2c1-c477-4271-af37-76ff2ea3e764','12ca67cd-7038-438f-acf0-d193a141a653','a1766688-7575-438d-8586-b0117a46153d','df47a465-f872-4968-a8e9-057059b8e900','14c58957-7396-4322-a237